# парсинг с незащищенного сайта

In [ ]:
#загружаем библиотеки
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import pandas as pd
import time

In [ ]:
#запихнули ссылку в перемнную
url = 'https://biggeek.ru/catalog/apple-iphone'

In [ ]:
resp = requests.get(url)

In [ ]:
resp

<Response [200]>

In [ ]:
soup = BeautifulSoup(resp.text)

In [ ]:
list_cards = soup.find_all(class_ = 'catalog-card')

In [ ]:
#сохранили 1 карточку в переменную
card = list_cards[0]

In [ ]:
#забрать текст из 1 карточки
card.text

'Смартфон Apple iPhone 14 Pro Max 128 ГБ (Тёмно-фиолетовый | Deep Purple) \n\nот 103 990 ₽129 990\n'

In [ ]:
#ищем отдельно название 1 товара
card.find_all(class_='catalog-card__title')

[<a class="catalog-card__title cart-modal-title" href="/products/apple-iphone-14-pro-max-128gb-deep-purple">Смартфон Apple iPhone 14 Pro Max 128 ГБ (Тёмно-фиолетовый | Deep Purple)</a>]

In [ ]:
#забрали название <a href="/products/apple-iphone-14-pro-max-128gb-deep-purple" class="catalog-card__title cart-modal-title">Смартфон Apple iPhone 14 Pro Max 128 ГБ (Тёмно-фиолетовый | Deep Purple)</a>
card_name = card.find_all(class_='catalog-card__title')[0].text

In [ ]:
#копируем href="/products/apple-iphone-14-pro-max-128gb-deep-purple"
card_url = card.find_all(class_='catalog-card__title')[0]['href']

In [ ]:
#забрали цену <b class="cart-modal-count">от 103 990 <span>₽</span></b>
card_price = card.find_all(class_='cart-modal-count')[0].text

In [ ]:
#для сбора данных с остальных карточек создадим список
total_list=[]
for card in list_cards:
  card_name = card.find_all(class_='catalog-card__title')[0].text
  card_url = card.find_all(class_='catalog-card__title')[0]['href']
  card_price = card.find_all(class_='cart-modal-count')[0].text

  temp_list = [card_name, card_url ,card_price]
  total_list.append(temp_list)

In [ ]:
#чтобы посмотреть скорость выполнения добавим tqdm
total_list=[]
for card in tqdm(list_cards):
  card_name = card.find_all(class_='catalog-card__title')[0].text
  card_url = card.find_all(class_='catalog-card__title')[0]['href']
  card_price = card.find_all(class_='cart-modal-count')[0].text

  temp_list = [card_name, card_url ,card_price]
  total_list.append(temp_list)

  0%|          | 0/24 [00:00<?, ?it/s]

In [ ]:
pd.set_option('display.max_rows',7)
pd.DataFrame(total_list, columns = ['card_name', 'card_url' ,'card_price'])

,card_name,card_url,card_price
0,Смартфон Apple iPhone 14 Pro Max 128 ГБ (Тёмно...,/products/apple-iphone-14-pro-max-128gb-deep-p...,от 103 990 ₽
1,Смартфон Apple iPhone 14 Pro 128 ГБ (Тёмно-фио...,/products/apple-iphone-14-pro-128gb-deep-purple,от 88 990 ₽
2,Смартфон Apple iPhone 14 Plus 128 ГБ (Синий | ...,/products/apple-iphone-14-plus-128gb-blue,от 69 990 ₽
...,...,...,...
21,Смартфон Apple iPhone 12 64 ГБ (Чёрный | Black),/products/apple-iphone-12-64gb-black,42 990 ₽
22,Смартфон Apple iPhone 12 mini 64 ГБ (Чёрный | ...,/products/apple-iphone-12-mini-64gb-black,49 990 ₽
23,Смартфон Apple iPhone SE 64 ГБ (Белый | White)...,/products/apple-iphone-se-2020-64gb-white,31 990 ₽


In [ ]:
#для сбора данных с остальных карточек на всех страницах доработаем список
#для этого нам надо скопировать url https://biggeek.ru/catalog/apple-iphone?page=2
#и доработать первую ссылку url = 'https://biggeek.ru/catalog/apple-iphone'
# вот так url = 'https://biggeek.ru/catalog/apple-iphone?page=1'
#строим полноценный алгоритм-пайплайн
url = 'https://biggeek.ru/catalog/apple-iphone?page=1'
resp = requests.get(url)
soup = BeautifulSoup(resp.text)
list_cards = soup.find_all(class_ = 'catalog-card')

total_list = []
for card in list_cards:
        card_name = card.find_all(class_='catalog-card__title')[0].text
        card_url = card.find_all(class_='catalog-card__title')[0]['href']
        card_price = card.find_all(class_='cart-modal-count')[0].text

        temp_list = [card_name, card_url ,card_price]
        total_list.append(temp_list)
pd.set_option('display.max_rows',7)
#создаем датафрейм
pd.DataFrame(total_list, columns = ['card_name', 'card_url' ,'card_price'])

,card_name,card_url,card_price
0,Смартфон Apple iPhone 14 Pro Max 128 ГБ (Тёмно...,/products/apple-iphone-14-pro-max-128gb-deep-p...,от 103 990 ₽
1,Смартфон Apple iPhone 14 Pro 128 ГБ (Тёмно-фио...,/products/apple-iphone-14-pro-128gb-deep-purple,от 88 990 ₽
2,Смартфон Apple iPhone 14 Plus 128 ГБ (Синий | ...,/products/apple-iphone-14-plus-128gb-blue,от 69 990 ₽
...,...,...,...
21,Смартфон Apple iPhone 12 64 ГБ (Чёрный | Black),/products/apple-iphone-12-64gb-black,42 990 ₽
22,Смартфон Apple iPhone 12 mini 64 ГБ (Чёрный | ...,/products/apple-iphone-12-mini-64gb-black,49 990 ₽
23,Смартфон Apple iPhone SE 64 ГБ (Белый | White)...,/products/apple-iphone-se-2020-64gb-white,31 990 ₽


In [ ]:
#оптимизируем код, тк страниц 12
total_list = []
for page in tqdm(range(1,13)):
    url = f'https://biggeek.ru/catalog/apple-iphone?page={page}'
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text)
    list_cards = soup.find_all(class_ = 'catalog-card')

    for card in list_cards:
        card_name = card.find_all(class_='catalog-card__title')[0].text
        card_url = card.find_all(class_='catalog-card__title')[0]['href']
        card_price = card.find_all(class_='cart-modal-count')[0].text

        temp_list = [card_name, card_url ,card_price]
        total_list.append(temp_list)

  0%|          | 0/12 [00:00<?, ?it/s]

In [ ]:
pd.set_option('display.max_rows',7)
pd.DataFrame(total_list, columns = ['card_name', 'card_url' ,'card_price'])

,card_name,card_url,card_price
0,Смартфон Apple iPhone 14 Pro Max 128 ГБ (Тёмно...,/products/apple-iphone-14-pro-max-128gb-deep-p...,от 103 990 ₽
1,Смартфон Apple iPhone 14 Pro 128 ГБ (Тёмно-фио...,/products/apple-iphone-14-pro-128gb-deep-purple,от 88 990 ₽
2,Смартфон Apple iPhone 14 Plus 128 ГБ (Синий | ...,/products/apple-iphone-14-plus-128gb-blue,от 69 990 ₽
...,...,...,...
275,Смартфон Apple iPhone 13 Pro 512 ГБ («Альпийск...,/products/apple-iphone-13-pro-512gb-alpine-green,111 990 ₽
276,Смартфон Apple iPhone 13 Pro Max 1 ТБ («Альпий...,/products/apple-iphone-13-pro-max-1tb-alpine-g...,124 990 ₽
277,Смартфон Apple iPhone 13 Pro 1 ТБ («Альпийский...,/products/apple-iphone-13-pro-1tb-alpine-green,118 990 ₽
